# Concurrency

- Concurrency Problem:
    - multiple transactions 
    - at the same time
    - that read/change 
    - the same shared data
- Solution:
    - Isolate our transaction from other transactions
- Different level of concurrency phenomena:
1. Dirty Reads:
    - Original balance account 5 = $35,000
    - After Transaction 1, the current balance = $30,000
    - But Transaction 1 is not committed or Rolled Back yet
    - Transaction 2 can see that the current balance = $30,000
    - However, Transaction 1 Rolled Back and current balance = $35,000
    - What Transaction 2 thinks is current balance = $30,000.
    - Transaction 2 read is a dirty read. 
2. Non-repeatable Reads:
    - Transaction 1 sees the current balance = $35,000
    - After Transaction 2, the current balance = $30,000
    - Transaction 2 committed the change
    - Transaction 1 sees the current balance = $30,000
    - Transaction 1 sees 2 different values in 2 different times
    - Problem : What happens if we make business decision based on first result ?
3. Pantom Reads:
    - One person starts Transaction 1 with balance between 45000 and 50000
    - Another person starts Transaction 2 with balance between 45000 and 50000
    - Transaction 2 commits the change
    - If we try to determine Transaction 1 with balance between 45000 and 50000, we also get Transaction 2
    - Our target was to get Transaction 1
    - But we get additional value of Transaction 2
    - The result is different from first one with extra information
    - Transaction 2 is Phantom Read (Extra row)
    - Problem : What happens if we make business decision based on first result ?


# Different Isolation Levels

- `READ UNCOMMITTED`
    - Lets to read rows modified by another transaction which hasn't been committed or rolled back yet
    - Can be faster
    - doesn't block other transactions
    - Allows dirty reads
    - Allows non-repeatable reads
    - Allows phantom reads
    - No blocking
    - Use when 
        - You don't mind concurrency phenomena.
        - You explicitly want to watch uncommitted data
- `READ COMMITTED` 
    - default
    - Does not Let to read rows modified by another transaction which hasn't been committed or rolled back yet (Has to wait until the first transaction is completed before starting another)
    - Prevents dirty reads
    - Allows non-repeatable reads
    - Allows phantom reads
    - No blocking
    - Use when 
        - You want to ensure that you only read committed data
        - You do not mind not non-repeatable and phantom reads
- `REPEATABLE READ`
    - Can't read uncommitted data from other transactions
    - If some data is read, other transactions cannot modify that data until REPEATABLE READ transaction finishes
    - Prevents dirty reads
    - Prevents other transactions from modifying the data you are reading, non-repeatable reads
    - Allows phantom reads
    - You can be blocked by a `REPEATABLE READ` transaction.
    - Use when 
        - You don't mind phantom reads.
        - You Only want to read committed data and don't want other transactions to modify what you are reading.
- `SERIALIZABLE`
    - Most restrictive isolation level
    - Prevents dirty reads
    - Locks record if index is provided with `WHERE` clause
    - Locks table if index is not provided.
    - Prevents non-repeatable reads
    - Prevents phantom reads
    - Good data consistency
    - You can be blocked by a `SERIALIZABLE` transaction
    - Use it when data consistency is a must
- `SNAPSHOT`
- example : `SET TRANSACTION ISOLATION LEVEL level_name`
- Know your isolation level:
```
SELECT 
CASE transaction_isolation_level
    WHEN 0 THEN 'UNSPECIFIED'
    WHEN 1 THEN 'READ UNCOMMITTED'
    WHEN 2 THEN 'READ COMMITTED'
    WHEN 3 THEN 'REPEATABLE READ '
    WHEN 4 THEN 'SERIALIZABLE'
    WHEN 5 THEN 'SNAPSHOT'
END AS transaction_isolation_level
FROM sys.dm_exec_sessions
WHERE session_id = @@SPID
```

# SNAPSHOT

- Every time a modification is made, previous version of row is stored in the `tempDB` database
- Only see committed changes that occurred before the start of the SNAPSHOT transaction and own changes
- Can't see any changes made by other transactions after the start of the SNAPSHOT transaction
- Readings don't block writings and writings don't block readings
- Can have update conflicts when updating same data at the same time
- Prevents dirty reads
- Prevents non-repeatable reads
- Prevents phantom reads
- Good data consistency
- NO BLOCKING of Reading or Writing
- `tempDB` increases
- Use When data consistency is a must and don't want blocks
- Enable:
    ```
    ALTER DATABASE myDatabaseName SET ALLOW_SNAPSHOT_ISOLATION ON;
    SET TRANSACTION ISOLATION LEVEL SNAPSHOT
    ```
<center><img src="images/04.jpg"  style="width: 400px, height: 300px;"/></center>

# READ COMMITTED SNAPSHOT

- Changes the behavior of `READ COMMITTED`
- makes every READ COMMITTED statement can only see committed changes that occurred before the START OF THE STATEMENT
- Can't have update conflicts
- `ALTER DATABASE myDatabaseName SET READ_COMMITTED_SNAPSHOT ON;`
    - `OFF` by default
-  the main difference between `READ COMMITTED SNAPSHOT` and the `SNAPSHOT` isolation level is that with the `SNAPSHOT` isolation level you can only see the committed changes that occur before the START OF A TRANSACTION and the changes made by that transaction AS OPPOSSED TO START OF THE STATEMENT
- Enable:
    ```
    ALTER DATABASE myDatabaseName SET ALLOW_SNAPSHOT_ISOLATION ON 
    ALTER DATABASE myDatabaseName SET READ_COMMITTED_SNAPSHOT ON.
    ```

# WITH (NOLOCK)

- Used to read uncommitted data 
- Unclock to only read
- `READ UNCOMMITTED` applies to the entire connection BUT `WITH (NOLOCK)` applies to a specific table
- Use under any isolation level when you just want to read uncommitted data from specific table
- Transaction 1:
    ```
    BEGIN TRAN 
    UPDATE accounts
    SET current_balance = 30000 WHERE account_id= 5;
    ```
- Transaction 2:
    ```
    SELECT current_balance FROM accounts
    WITH (NOLOCK)
    WHERE account_id = 5;
    ```